In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Controlled Generation with the Gemini API

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fcontrolled-generation%2Fintro_controlled_generation.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/controlled-generation/intro_controlled_generation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://goo.gle/3Pyftqr">
      <img width="32px" src="https://cdn.qwiklabs.com/assets/gcp_cloud-e3a77215f0b8bfa9b3f611c0d2208c7e8708ed31.svg" alt="Google Cloud logo"><br> Open in  Skills
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| Author |
| --- |
| [Eric Dong](https://github.com/gericdong) |

## Overview

The notebook provides a tutorial on using the Gemini API's controlled generation capabilities to generate structured output. This is important for application reliability, as it ensures the model's response is in a predictable format like JSON, which is easier to process than free-form text.

Learn more about [structured output](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output).


### Objectives

In this tutorial, you learn how to use the controlled generation capability in the Gemini API in Vertex AI to generate model responses in a structured data format.

You will complete the following tasks:

- Generate JSON
    - Option 1: Using Pydantic models
    - Option 2: Using OpenAPI Schema
    - Option 3: Using JSON Schema
- Generate Enum values
- Apply controlled generation to several use cases

## Get started

### Install Google Gen AI SDK for Python

In [2]:
%pip install --upgrade --quiet google-genai

Note: you may need to restart the kernel to use updated packages.


### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
from google import genai

PROJECT_ID = "qwiklabs-gcp-00-1039a9c9d7ae"
LOCATION = "global"

### Import libraries

In [4]:
from enum import Enum

from google import genai
from google.genai.types import GenerateContentConfig, Part, SafetySetting
from pydantic import BaseModel

### Connect to the generative AI service on Vertex AI

In [5]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Supported Models

You can use this feature in the [supported models](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output). The following examples use Gemini 2.5 Flash (`gemini-2.5-flash`).

In [6]:
MODEL_ID = "gemini-2.5-flash" # @param {type:"string"}

## Generating JSON

The Gemini models allow you to define a response schema to specify the structure of a model's output, the field names, and the expected data type for each field. The response schema is specified in the `response_schema` or the `response_json_schema` parameter in `config`, and the model output will strictly follow that schema.

You can configure response schema using the following options:

- Using Pydantic models with `response_schema`
- Using OpenAPI Schema with `response_schema`
- Using JSON Schema with `response_json_schema`


### **Option 1**: Using Pydantic models

You can provide the schemas as [Pydantic](https://docs.pydantic.dev/) models or a [JSON](https://www.json.org/json-en.html) string and the model will respond as JSON or an [Enum](https://docs.python.org/3/library/enum.html) depending on the value set in `response_mime_type`.

When prompting the model to generate the content, pass the schema to the `response_schema` field of the `generation_config`.

You also need to specify the model output format in the `response_mime_type` field. Output formats such as `application/json` and `text/x.enum` are supported.

In [7]:
class CountryInfo(BaseModel):
    name: str
    population: int
    capital: str
    continent: str
    gdp: int
    official_language: str
    total_area_sq_mi: int


response = client.models.generate_content(
    model=MODEL_ID,
    contents="Fetch key facts for the United States.",
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=CountryInfo,
    ),
)

print(response.text)

{
  "name": "United States",
  "population": 334805269,
  "capital": "Washington, D.C.",
  "continent": "North America",
  "gdp": 27970000000000,
  "official_language": "English",
  "total_area_sq_mi": 3796742
}


You can either parse the response string as JSON, or use the `parsed` field to get the response as an object or dictionary.

In [8]:
country: CountryInfo = response.parsed
print(country.name)
print(country.population)
print(country.capital)

United States
334805269
Washington, D.C.


### **Option 2**: Using OpenAPI Schema

You also can define a response schema using a subset of the OpenAPI schema. The supported fields are listed below. All other fields are ignored.

- `enum`
- `items`
- `maxItems`
- `nullable`
- `properties`
- `required`

By default, fields are optional, meaning the model can populate the fields or skip them. You can set fields as required to force the model to provide a value. If there's insufficient context in the associated input prompt, the model generates responses mainly based on the data it was trained on.

In [9]:
response_schema = {
    "type": "OBJECT",
    "properties": {
        "name": {"type": "STRING"},
        "population": {"type": "INTEGER"},
        "capital": {"type": "STRING"},
        "continent": {"type": "STRING"},
        "gdp": {"type": "INTEGER"},
        "official_language": {"type": "STRING"},
        "total_area_sq_mi": {"type": "INTEGER"},
    },
    "required": [
        "name",
        "population",
        "capital",
        "continent",
        "gdp",
        "official_language",
        "total_area_sq_mi",
    ],
}

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Fetch key facts for Canada.",
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

print(response.text)

{
  "name": "Canada",
  "population": 40000000,
  "capital": "Ottawa",
  "continent": "North America",
  "gdp": 2139000000000,
  "official_language": "English and French",
  "total_area_sq_mi": 3855100
}


### **Option 3**: Using JSON Schema

[JSON Schema](https://json-schema.org/) is a standard for describing the structure and validation rules for JSON data.

Support for the JSON Schema standard is available in preview. You pass the schema to the `response_json_schema` parameter.

In [10]:
# An advanced JSON Schema with conditional logic
country_info_schema = {
    "type": "object",
    # 1. Define ALL possible properties at the top level.
    "properties": {
        "country_name": {
            "type": "string",
            "description": "The official name of the country.",
        },
        "capital": {
            "type": "string",
            "description": "The capital city of the country.",
        },
        "is_g7_member": {
            "type": "boolean",
            "description": "True if the country is a member of the G7.",
        },
        "government_type": {
            "type": "string",
            "enum": ["Republic", "Constitutional Monarchy"],
            "description": "The type of government.",
        },
        "head_of_state": {
            "type": "string",
            "description": "The name of the current President or head of state (for Republics).",
        },
        "monarch": {
            "type": "string",
            "description": "The name of the current monarch (for Monarchies).",
        },
        "g7_join_year": {
            "type": "integer",
            "description": "The year the country joined the G7 (if applicable).",
        },
    },
    # 2. List only the properties that are ALWAYS required.
    "required": ["country_name", "capital", "is_g7_member", "government_type"],
    # 3. Use allOf to layer on the conditional requirements.
    "allOf": [
        {
            "if": {"properties": {"government_type": {"const": "Republic"}}},
            "then": {"required": ["head_of_state"]},
        },
        {
            "if": {
                "properties": {"government_type": {"const": "Constitutional Monarchy"}}
            },
            "then": {"required": ["monarch"]},
        },
        {
            "if": {"properties": {"is_g7_member": {"const": True}}},
            "then": {"required": ["g7_join_year"]},
        },
    ],
}

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Fetch key facts for the United Kingdom.",
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_json_schema=country_info_schema,
    ),
)

print(response.text)

{
  "country_name": "United Kingdom",
  "capital": "London",
  "is_g7_member": true,
  "government_type": "Constitutional Monarchy",
  "g7_join_year": 1975,
  "monarch": "King Charles III"
}


## Generating Enum Values

To force the model to choose from a predefined list of options, you can use an
enum. This ensures data consistency by preventing unpredictable responses.

To generate an enum, set the `response_mime_type` field to `text/x.enum`.


In [11]:
class InstrumentEnum(Enum):
    PERCUSSION = "Percussion"
    STRING = "String"
    WOODWIND = "Woodwind"
    BRASS = "Brass"
    KEYBOARD = "Keyboard"


response = client.models.generate_content(
    model=MODEL_ID,
    contents="What instrument plays multiple notes at once?",
    config=GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=InstrumentEnum,
    ),
)

print(response.text)

Keyboard


## Use Case Examples

Controlled generation can be used to ensure that model outputs adhere to a specific structure (e.g., JSON), instruct the model to perform pure multiple choices (e.g., sentiment classification), or follow certain style or guidelines.

Let's use controlled generation in the following use cases that require output constraints.

### **Example**: Generate game character profile

In this example, you instruct the model to create a game character profile with some specific requirements, and constraint the model output to a structured format. This example also demonstrates how to configure the `response_schema` and `response_mime_type` fields in `config` in conjunction with `safety_settings`.

In [14]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "name": {"type": "STRING"},
            "age": {"type": "INTEGER"},
            "occupation": {"type": "STRING"},
            "background": {"type": "STRING"},
            "playable": {"type": "BOOLEAN"},
            "children": {
                "type": "ARRAY",
                "items": {
                    "type": "OBJECT",
                    "properties": {
                        "name": {"type": "STRING"},
                        "age": {"type": "INTEGER"},
                    },
                    "required": ["name", "age"],
                },
            },
        },
        "required": ["name", "age", "occupation", "children"],
    },
}

prompt = """
    Generate a character profile for a video game, including the character's name, age, occupation, background, names of their
    three children, and whether they can be controlled by the player.
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
        safety_settings=[
            SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT",
                threshold="BLOCK_LOW_AND_ABOVE",
            ),
            SafetySetting(
                category="HARM_CATEGORY_HARASSMENT",
                threshold="BLOCK_LOW_AND_ABOVE",
            ),
            SafetySetting(
                category="HARM_CATEGORY_HATE_SPEECH",
                threshold="BLOCK_LOW_AND_ABOVE",
            ),
            SafetySetting(
                category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                threshold="BLOCK_LOW_AND_ABOVE",
            ),
        ],
    ),
)

character: dict = response.parsed
print(character)

[{'name': 'Elara Vance', 'age': 35, 'occupation': 'Starship Mechanic', 'background': 'Born on a desolate mining colony, Elara learned to fix anything with an engine from a young age. She dreams of building a vessel capable of interstellar travel for her family.', 'playable': True, 'children': [{'name': 'Kael Vance', 'age': 10}, {'name': 'Lyra Vance', 'age': 7}, {'name': 'Jor Vance', 'age': 3}]}]


### **Example**: Extract errors from log data

In this example, you use the model to pull out specific error messages from unstructured log data, extract key information, and constraint the model output to a structured format.

Some properties are set to nullable so the model can return a null value when it doesn't have enough context to generate a meaningful response.


In [13]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "timestamp": {"type": "STRING"},
            "error_code": {"type": "INTEGER", "nullable": True},
            "error_message": {"type": "STRING"},
        },
        "required": ["timestamp", "error_message", "error_code"],
    },
}

prompt = """
[15:43:28] ERROR: Could not process image upload: Unsupported file format. (Error Code: 308)
[15:44:10] INFO: Search index updated successfully.
[15:45:02] ERROR: Service dependency unavailable (payment gateway). Retrying... (Error Code: 5522)
[15:45:33] ERROR: Application crashed due to out-of-memory exception. (Error Code: 9001)
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

log_data: dict = response.parsed
print(log_data)

[{'timestamp': '15:43:28', 'error_code': 308, 'error_message': 'Could not process image upload: Unsupported file format.'}, {'timestamp': '15:44:10', 'error_code': None, 'error_message': 'Search index updated successfully.'}, {'timestamp': '15:45:02', 'error_code': 5522, 'error_message': 'Service dependency unavailable (payment gateway). Retrying...'}, {'timestamp': '15:45:33', 'error_code': 9001, 'error_message': 'Application crashed due to out-of-memory exception.'}]


### **Example**: Analyze product review data

In this example, you instruct the model to analyze product review data, extract key entities, perform sentiment classification (multiple choices), provide additional explanation, and output the results in JSON format.

In [15]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "rating": {"type": "INTEGER"},
                "flavor": {"type": "STRING"},
                "sentiment": {
                    "type": "STRING",
                    "enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"],
                },
                "explanation": {"type": "STRING"},
            },
            "required": ["rating", "flavor", "sentiment", "explanation"],
        },
    },
}

prompt = """
  Analyze the following product reviews, output the sentiment classification and give an explanation.

  - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
  - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

print(response.text)

[
  [
    {
      "rating": 4,
      "flavor": "Strawberry Cheesecake",
      "sentiment": "POSITIVE",
      "explanation": "The review uses strong positive language like 'Absolutely loved it!' and 'Best ice cream I've ever had', clearly indicating a very favorable opinion."
    }
  ],
  [
    {
      "rating": 1,
      "flavor": "Mango Tango",
      "sentiment": "NEGATIVE",
      "explanation": "Despite starting with 'Quite good', the phrase 'but a bit too sweet for my taste' along with the low rating of 1 indicates a negative overall experience for the reviewer regarding the sweetness of the product."
    }
  ]
]


### **Example**: Detect objects in images

You can also use controlled generation in multimodality use cases. In this example, you instruct the model to detect objects in the images and output the results in JSON format. These images are stored in a Google Storage bucket.

- [office-desk.jpeg](https://storage.googleapis.com/cloud-samples-data/generative-ai/image/office-desk.jpeg)
- [gardening-tools.jpeg](https://storage.googleapis.com/cloud-samples-data/generative-ai/image/gardening-tools.jpeg)

In [16]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "object": {"type": "STRING"},
            },
        },
    },
}

prompt = "Generate a list of objects in the images."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/image/office-desk.jpeg",
            mime_type="image/jpeg",
        ),
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/image/gardening-tools.jpeg",
            mime_type="image/jpeg",
        ),
        prompt,
    ],
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

object_list = response.parsed
print(object_list)

[[{'object': 'globe'}, {'object': 'Eiffel Tower statue'}, {'object': 'airplane model'}, {'object': 'passport'}, {'object': 'sunglasses'}, {'object': 'dollar bills'}, {'object': 'notebook'}, {'object': 'pen'}, {'object': 'tablet'}, {'object': 'wireless keyboard'}, {'object': 'wireless mouse'}, {'object': 'mini shopping cart with gifts'}, {'object': 'coffee cup'}], [{'object': 'watering can'}, {'object': 'potted plant'}, {'object': 'empty terracotta pot'}, {'object': 'gardening gloves'}, {'object': 'hand rake'}, {'object': 'small shovel'}]]


### **Example**: Respond with a single plain text enum value

This example identifies the genre of a movie based on its description. The output is one plain-text enum value that the model selects from a list of values that are defined in the response schema. Note that in this example, the `response_mime_type` field is set to `text/x.enum`.


In [17]:
response_schema = {"type": "STRING", "enum": ["drama", "comedy", "documentary"]}

prompt = (
    "The film aims to educate and inform viewers about real-life subjects, events, or people."
    "It offers a factual record of a particular topic by combining interviews, historical footage, "
    "and narration. The primary purpose of a film is to present information and provide insights "
    "into various aspects of reality."
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=response_schema,
    ),
)

print(response.text)

documentary


### **Example**: Use a JSON schema with conditional logic

This example demonstrates how to use a JSON schema with conditional logic to guide the model to generate highly structured data. The schema defines a format for order status updates:

- Every update includes an order_id and a status.
- If the status is `SHIPPED`, a tracking_number is required.
- If the status is `DELIVERED`, a delivery_date is required.

The prompt is a simple request for three different order updates. By using the provided JSON schema in the request, the model is forced to generate a structured JSON array that includes the conditional fields for each order, resulting in a predictable JSON output.

In [ ]:
# A JSON schema for order status updates with conditional fields
order_schema = {
    "type": "object",
    # 1. Define all possible properties that could appear.
    "properties": {
        "order_id": {
            "type": "string",
        },
        "status": {
            "type": "string",
            "enum": ["PENDING", "SHIPPED", "DELIVERED"],
        },
        "tracking_number": {
            "type": "string",
        },
        "delivery_date": {
            "type": "string",
            "format": "date",
        },
    },
    # 2. List only the properties that are ALWAYS required.
    "required": ["order_id", "status"],
    # 3. Add the conditional logic.
    "allOf": [
        {
            "if": {"properties": {"status": {"const": "SHIPPED"}}},
            "then": {"required": ["tracking_number"]},
        },
        {
            "if": {"properties": {"status": {"const": "DELIVERED"}}},
            "then": {"required": ["delivery_date"]},
        },
    ],
}

# A prompt asking for updates on three different orders
prompt = """
Provide a status update for the following orders:
- Order 98765 is still processing.
- Order 54321 was shipped. (Tracking number: TRK54321XYZ)
- Order 12345 was delivered on July 31, 2025.
"""

# The API call using the JSON schema
response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",  # required
        response_json_schema={"type": "array", "items": order_schema},
    ),
)

print(response.text)